In [1]:
import numpy as np
import cv2
import math

In [2]:
#reading the image as gray scale. Image is present in the same folder as this file
img = cv2.imread('keypoint.jpg', 0)
[rows,cols] = img.shape

In [3]:
#padding
def padding(img):
    listofzeros = [0] * (cols+6)
    image_pad_x = list()
    for i in range(3):
        image_pad_x.append(listofzeros)
    for i in img:
        i = list(i)
        i.insert(0,0)
        i.insert(1,0)
        i.insert(2,0)
        i.insert(len(i),0)
        i.insert(len(i)+1,0)
        i.insert(len(i)+2,0)
        image_pad_x.append(i)
    for i in range(3):
        image_pad_x.append(listofzeros)
    image_pad_x = np.asarray(image_pad_x)
    return image_pad_x

In [4]:
#calculating the gaussian function
def gaussian_function(sigma,x,y):
    val = (1/(2*math.pi*sigma*sigma))*(math.exp(-((x*x)+(y*y))/(2*sigma*sigma)))
    return val

In [5]:
#computing the gaussian kernel
def gaussian_kernel(sigma):
    sum_val=0
    gaussian_kernel_array = [[0.0] * 7 for _ in range(7)]
    len(gaussian_kernel_array)
    k=0
    for i in range(-3,4):
        l=0
        for j in range(-3,4):
            gaussian_kernel_array[k][l] = gaussian_function(sigma,i,j)
            l = l+1
        k = k+1
    #normalisation
    maximum = gaussian_kernel_array[i][j]
    for i in range(7):
        for j in range(7):
            if maximum < abs(gaussian_kernel_array[i][j]):
                maximum = abs(gaussian_kernel_array[i][j])
    for i in range(7):
        for j in range(7):
            gaussian_kernel_array[i][j] = abs(gaussian_kernel_array[i][j])/maximum
              
    gaussian_kernel_array =np.asarray(gaussian_kernel_array)
    return gaussian_kernel_array

In [30]:
#convolution
def convolution(image,kernel):
    [rows,cols] = image.shape
    image_pad_x = padding(image)
    #[rows_pad,cols_pad] = image_pad_x.shape
    oct_img = [[0.0] * (int)(cols) for _ in range((int)(rows))]
    for i in range(3,rows-3):
        for j in range(3,cols-3):
            s=0
            for k in range(7):
                for l in range(7):
                    s+=image[i-3+k,j-3+l]*kernel[k,l]
            oct_img[i-3][j-3]=s
    #normalisation
    maximum = abs(oct_img[0][0]) 
    for i in range(rows):
        for j in range(cols):
            if maximum < abs(oct_img[i][j]):
                    maximum = abs(oct_img[i][j])  
    for i in range(rows):
        for j in range(cols):
            oct_img[i][j] = abs(oct_img[i][j])*255/maximum
    oct_img = np.asarray(oct_img)
    return oct_img

In [34]:
#image resizing to generate consequent octaves
def resize(img,ctr):
    [rows,cols] = img.shape
    resized_img = [[0.0] * (int)(cols/ctr) for _ in range((int)(rows/ctr))]
    resized_img = np.asarray(resized_img)
    k=l=0
    for i in range(0,rows,ctr):
        l=0
        for j in range(0,cols,ctr):
            resized_img[k,l] = img[i,j]
            l = l+1
        k= k+1
    return resized_img

In [35]:
#Computing the Difference of Gaussian
def compute_DoG(octave_1,octave_2):
    oct_1 = np.asarray(octave_1)
    oct_2 = np.asarray(octave_2)
    return (oct_1 - oct_2)

In [36]:
sqrt_2 = math.sqrt(2)
sigma = [[1/sqrt_2, 1, sqrt_2, 2, 2*sqrt_2],
         [sqrt_2, 2, 2*sqrt_2, 4, 4*sqrt_2],
         [2*sqrt_2, 4, 4*sqrt_2, 8, 8*sqrt_2],
         [4*sqrt_2, 8, 8*sqrt_2, 16, 16*sqrt_2]]
resize_ctr = [1,2,4,8]

In [37]:
#1st Octave
octave_1 = list()
resized_img = resize(img,1)
for j in range(5):
        gaussian_kernel_array = gaussian_kernel(sigma[0][j])
        octave= convolution(resized_img,gaussian_kernel_array)
        file_name = "Octave_1_Scale_"+str(j)+".png"
        cv2.imwrite(file_name,np.asarray(octave))
        octave_1.append(octave)
 

In [38]:
#2nd Octave
octave_2 = list()
resized_img = resize(img,2)
for j in range(5):
        gaussian_kernel_array = gaussian_kernel(sigma[1][j])
        octave= convolution(resized_img,gaussian_kernel_array)
        file_name = "Octave_2_Scale_"+str(j)+".png"
        cv2.imwrite(file_name,np.asarray(octave))
        octave_2.append(octave)

In [33]:
#3rd Octave
octave_3 = list()
resized_img = resize(img,4)
for j in range(5):
        gaussian_kernel_array = gaussian_kernel(sigma[2][j])
        octave= convolution(resized_img,gaussian_kernel_array)
        file_name = "Octave_3_Scale_"+str(j)+".png"
        cv2.imwrite(file_name,np.asarray(octave))
        octave_3.append(octave)

In [14]:
#4th Octave
octave_4 = list()
resized_img = resize(img,8)
for j in range(5):
        gaussian_kernel_array = gaussian_kernel(sigma[2][j])
        octave= convolution(resized_img,gaussian_kernel_array)
        file_name = "Octave_4_Scale_"+str(j)+".png"
        cv2.imwrite(file_name,np.asarray(octave))
        octave_4.append(octave)

In [15]:
def FindKeyPoint(DoG1,DoG2,DoG3):
    X = list()
    KeyPoint = list()
    for i in range(1,DoG1.shape[0]-1):
        for j in range(1,DoG1.shape[1]-1):
            x1 = DoG1[i-1:i+2,j-1:j+2]
            x2 = DoG2[i-1:i+2,j-1:j+2]
            x3 = DoG3[i-1:i+2,j-1:j+2]
            KeyPointx = i
            KeyPointy = j
            X = []
            for k in range(3):
                for l in range(3):
                        X.append(x1[k,l])
                        X.append(x2[k,l])
                        X.append(x3[k,l])
            Min = min(X)        
            Max = max(X)
            #print(Min,Max,DoG2[i+1,j+1])
            if x2[1,1]==Min or x2[1,1]==Max and x2[1,1]>0.03 :
                KeyPoint.append([KeyPointx,KeyPointy])
    return KeyPoint

In [16]:
def normalise_DoG_to_Show(dog):
    maximum = dog[0][0]
    for i in range(len(dog)):
        for j in range(len(dog[i])):
            if maximum < abs(dog[i][j]):
                maximum = abs(dog[i][j])
    for i in range(len(dog)):
        for j in range(len(dog[i])):
            dog[i][j]= abs(dog[i][j])*255/maximum
    return dog

In [17]:
#1st octave
DoG1 = list()
for i in range(len(octave_1)-1):
    DoG1_val = compute_DoG(octave_1[i],octave_1[i+1])
    #print(DoG1)
    DoG1.append(DoG1_val)
    filename = "DoG_1."+str(i)+".png"
    #norm_DoG1 = normalise_DoG_to_Show(DoG1_val)
    #cv2.imwrite(filename,np.asarray(norm_DoG1))
    cv2.imshow(filename,np.asarray(DoG1_val))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
#2nd octave
DoG2 = list()
for i in range(0,len(octave_2)-1):
    DoG2_val = compute_DoG(octave_2[i],octave_2[i+1])
    #norm_DoG2 = normalise_DoG_to_Show(DoG2_val)
    filename = "DoG_2."+str(i)+".png"
    #cv2.imwrite(filename,np.asarray(norm_DoG2))
    cv2.imshow(filename,np.asarray(DoG2_val))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    DoG2.append(DoG2_val)

In [19]:
#3rd octave
DoG3 = list()
for i in range(0,len(octave_3)-1):
    DoG3_val = compute_DoG(octave_3[i],octave_3[i+1])
    #norm_DoG3 = normalise_DoG_to_Show(DoG3_val)
    filename = "DoG_3."+str(i)+".png"
    #cv2.imwrite(filename,np.asarray(norm_DoG3))
    cv2.imshow(filename,np.asarray(DoG3_val))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    DoG3.append(DoG3_val)

In [20]:
#4th octave
DoG4 = list()
for i in range(0,len(octave_4)-1):
    DoG4_val = compute_DoG(octave_4[i],octave_4[i+1])
    #norm_DoG4 = normalise_DoG_to_Show(DoG4_val)
    filename = "DoG_4."+str(i)+".png"
    #cv2.imwrite(filename,np.asarray(norm_DoG4))
    cv2.imshow(filename,np.asarray(DoG4_val))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    DoG4.append(DoG4_val)

In [21]:
#To display the keypoint on the image
def ShowKeyPoint(img,KeyPoint,ctr):
    [rows,cols] = img.shape
    for i in range(len(KeyPoint)):
        p = KeyPoint[i]
        img[p[0]*ctr,p[1]*ctr] = 255
    return (img)

In [22]:
#keypoint detection
#keypoint for DoG1
print("keypoint for DoG1")
keyPoint1 = list()
for i in range(1,len(DoG1)-1):
    keyPoint1 = FindKeyPoint(DoG1[i-1],DoG1[i],DoG1[i+1])
    print(len(keyPoint1))
    filename = "DoG1_keypoint_new"+str(i)+".png"
    show1 = ShowKeyPoint(img,keyPoint1,1)
    #cv2.imwrite(filename,show1)
    cv2.imshow(filename,show1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

keypoint for DoG1
6684
5848


In [28]:
#keypoint for DoG2
print("keypoint for DoG2")
img = cv2.imread("keypoint.jpg",0)
keyPoint2 = list()
for i in range(1,len(DoG2)-1):
    keyPoint2 = FindKeyPoint(DoG2[i-1],DoG2[i],DoG2[i+1])
    print(len(keyPoint2))
    filename = "DoG2_keypoint_"+str(i)+".png"
    show2 = ShowKeyPoint(img,keyPoint2,2)
    cv2.imwrite(filename,show2)
    '''cv2.imshow(filename,show2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''

keypoint for DoG2
2911
2934


In [29]:
#keypoint for DoG3
print("keypoint for DoG3")
img = cv2.imread("keypoint.jpg",0)
for i in range(1,len(DoG3)-1):
    keyPoint3 = FindKeyPoint(DoG3[i-1],DoG3[i],DoG3[i+1])
    print(len(keyPoint3))
    filename = "DoG3_keypoint_"+str(i)+".png"
    show3 = ShowKeyPoint(img,keyPoint3,4)
    cv2.imwrite(filename,show3)
    '''cv2.imshow(filename,show3)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''

keypoint for DoG3
1270
1260


In [ ]:
#keypoint for DoG4
print("keypoint for DoG4")
img = cv2.imread("keypoint.jpg",0)
for i in range(1,len(DoG4)-1):
    keyPoint4 = FindKeyPoint(DoG4[i-1],DoG4[i],DoG4[i+1])
    print(len(keyPoint4))
    filename = "DoG4_keypoint_"+str(i)+".png"
    show4 = ShowKeyPoint(img,keyPoint4,8)
    cv2.imwrite(filename,show4)
    '''cv2.imshow(filename,show4)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''